## **silver-companies-information**
<p>This notebook collects and stores basic company metadata from Yahoo Finance, such as sector, industry, market cap, and location. It acts as a reference layer for enriching financial and news datasets with company-level context.</p>

#### **Importing necessary libraries**

In [56]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import DeltaTable
import datetime
import traceback

StatementMeta(, f7878fb6-780a-4c94-9699-752618c12d9f, 60, Finished, Available, Finished)

#### **Variables**

In [57]:
file_name = "comp_info_data"
layer_name = "bronze"

StatementMeta(, f7878fb6-780a-4c94-9699-752618c12d9f, 61, Finished, Available, Finished)

#### **Step 2: Read the Stock Companies Information Data**

In [58]:
com_codes = spark.read.csv("Files/master_files/company_code.csv", header=True)
com_info_df = spark.read.parquet(f"Files/{layer_name}/{file_name}/")

StatementMeta(, f7878fb6-780a-4c94-9699-752618c12d9f, 62, Finished, Available, Finished)

#### **Step 3: Select the necessary columns, replacing the 'NaN' values, change the date columns type in the Companies Information Data**

In [59]:
def replace_nan_with_null(df):
    """
    For each column in df:
    - Replace string 'nan' (case-insensitive) with null.
    - Replace numeric NaN with null.
    """
    new_cols = []
    
    for field in df.schema.fields:
        colname = field.name
        dtype = field.dataType
        
        c = col(colname)
        
        if isinstance(dtype, StringType):
            new_c = when(
                lower(c) == 'nan',
                None
            ).otherwise(c)
            
        elif isinstance(dtype, (DoubleType, FloatType, DecimalType)):
            new_c = when(
                isnan(c),
                None
            ).otherwise(c)
            
        else:
            new_c = c
        
        new_cols.append(new_c.alias(colname))
    
    return df.select(*new_cols)

StatementMeta(, f7878fb6-780a-4c94-9699-752618c12d9f, 63, Finished, Available, Finished)

In [60]:
output_df = replace_nan_with_null(com_info_df)

StatementMeta(, f7878fb6-780a-4c94-9699-752618c12d9f, 64, Finished, Available, Finished)

In [61]:
final_output_df = output_df\
                .withColumn('dividendDate',date_format(from_unixtime(col('dividendDate')),'yyyy-MM-dd'))\
                .join(com_codes, output_df.Company == com_codes.short_code, 'left')\
                .select(
                    col('Company').alias('company_name'),
                    col('comp_url').alias('company_image'),
                    col('shortName').alias('short_name'),
                    col('address1').alias('address'),
                    'city',
                    'state',
                    'zip',
                    'country',
                    'phone',
                    'website',
                    'industry',
                    'sector',
                    col('recommendationKey').alias('recommendation_key'),
                    col('financialCurrency').alias('financial_currency'),
                    col('customPriceAlertConfidence').alias('custom_price_alert_confidence'),
                    'exchange',
                    col('fullExchangeName').alias('full_exchange_name'),
                    col('dividendDate').alias('dividend_date'),
                    col('ipoExpectedDate').alias('ipo_expected_date'),
                    col('fullTimeEmployees').alias('full_time_employees'),
                    col('marketCap').alias('market_cap'),
                    col('auditRisk').alias('audit_risk'),
                    col('boardRisk').alias('board_risk'),
                    col('compensationRisk').alias('compensation_eisk'),
                    col('shareHolderRightsRisk').alias('share_holder_rights_risk'),
                    col('overallRisk').alias('overall_risk')
                )\
                .filter(col('short_name').isNotNull())



StatementMeta(, f7878fb6-780a-4c94-9699-752618c12d9f, 65, Finished, Available, Finished)

#### **Step 4: Storing the transformed data into a Delta table in the Silver layer**

In [62]:
delta_path = f"Files/silver/{file_name}"

try:
    if DeltaTable.isDeltaTable(spark, delta_path):
        delta_table = DeltaTable.forPath(spark, delta_path)
        
        delta_table.alias("target").merge(
            final_output_df.alias("source"),
            (col("target.company_name") == col("source.company_name"))
        ).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

    else:
        final_output_df.write.format("delta").mode("overwrite").save(delta_path)

except Exception as e:
    print("An error occurred during the processing:")
    traceback.print_exc()

StatementMeta(, f7878fb6-780a-4c94-9699-752618c12d9f, 66, Finished, Available, Finished)